# Regime Predictor

Rewriting the PCA-based regime projection from the ML and RL in Finance course by Igor Halperin. Theory is based on 
*Principal Components as a Measure of Systemic Risk* by *Mark Kritzman, Yuanzhen Li, Sebastien Page, and Roberto Rigobon*

In [ ]:
import glob
import pandas as pd
import numpy as np
import sklearn.decomposition
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 13,8
import seaborn as sns
import pandas as pd

#### Dataset:  daily prices of for market representative such as S&P 500, MSCI World, etc. ####

In [ ]:
path = r'./individual_stocks/' # use your path
all_files = glob.glob(path + "*.csv")

li = []
cols = []
for filename in all_files[:]:
    df = pd.read_csv(filename, index_col=0, header=0, date_parser=lambda dt: pd.to_datetime(dt, format='%Y-%m-%d'))
    df = df.Close
    cols.append(filename.partition("_")[-1].partition("_")[0].strip("stocks/"))
    li.append(df)

frame = pd.concat(li, axis=1, ignore_index=True)
frame = frame.rename(columns=dict(zip(range(len(all_files)), cols)))
asset_prices = frame.interpolate().fillna(method="ffill").fillna(method="bfill")
print('Asset prices shape', asset_prices.shape)
display(asset_prices.iloc[:, :n_stocks_show].head(-3))
asset_prices.index

### Calculate daily log-returns

In [ ]:
asset_returns = np.log(asset_prices) - np.log(asset_prices.shift(1))
# asset_returns = asset_prices.pct_change(periods=1) #log(1+r) approx r
asset_returns = asset_returns.iloc[1:, :]
asset_returns.iloc[:, :n_stocks_show].head()

In [ ]:
def normalize_returns(df):
    """
    Normalize, i.e. center and divide by standard deviation raw asset returns data

    Arguments:
    r_df -- a pandas.DataFrame of asset returns

    Return:
    normed_df -- normalized returns
    """
    return (df - df.mean(axis=0))/df.std(axis=0)

In [ ]:
normed_r = normalize_returns(asset_returns)
display(normed_r.iloc[:, :n_stocks_show].head(-3))
#easier than standardscaler from sklearn

Now we compute the Absorption Ratio(AR) by taking a moving window of the returns for which we compute the covariance matrix and do a PCA. The data is based on step_size (business daily, ...) Then we look, how much variation of the complete market is determined already by 1/5 of the constitutents.

In [ ]:
def absorption_ratio(explained_variance, n_components):
    """
    Calculate absorption ratio via PCA.
    
    Arguments:
    explained_variance -- 1D np.array of explained variance by each pricincipal component, in descending order
    n_components -- an integer, a number of principal components to compute absorption ratio
    Return:
    ar -- absorption ratio
    """
    ar = np.sum(explained_variance[:n_components]) / np.sum(explained_variance)
    return ar

In [ ]:
stock_tickers = asset_returns.columns.values[:-1]

lookback_window = 252 * 2   # in (days)
num_assets = len(stock_tickers)
step_size = 1          # days : 5 - weekly, 21 - monthly, 63 - quarterly

# fix 20% of principal components for absorption ratio calculation. How much variance do they explain?
absorb_comp = int((1 / 5) * num_assets)  

print('Lookback window = %d' % lookback_window)
print('Step size = %d' % step_size)
print('Number of stocks = %d' % num_assets)
print('Number of principal components = %d' % absorb_comp)

# indexes date on which to compute PCA
days_offset = 4 * 252
num_days = 12 * 252 + days_offset
pca_ts_index = normed_r.index[list(range(lookback_window + days_offset, min(num_days, len(normed_r)), step_size))]

# allocate array for storing absorption ratio
absorp_ratio = np.array([np.nan]*len(pca_ts_index))

assert 'SPX' not in normed_r.iloc[:lookback_window, :-1].columns.values, "By accident included SPX index"

In [ ]:
#%%timeit
ik = 0
for ix in range(lookback_window + days_offset, min(num_days, len(normed_r)), step_size):
    ret_frame = normed_r.iloc[(ix - lookback_window) : ix, :-1]  # fixed window
    cov_mat = ret_frame.cov()
    
    if ik == 0 or ik % 21 == 0: # to speed up convergence compute just once a month?
        pca = sklearn.decomposition.PCA() #restrict to n_components=absorb_comp and rely on the ratio
        pca.fit(cov_mat)
        #absorp_ratio[ik]= pca.explained_variance_ratio_.sum()
        absorp_ratio[ik] = absorption_ratio(pca.explained_variance_, absorb_comp) 
    else:
        absorp_ratio[ik] = absorp_ratio[ik-1]       
    ik += 1

ts_absorb_ratio = pd.Series(absorp_ratio, index=pca_ts_index)

In [ ]:
#how much variance do the fix 20% of principal components explain?
sns.lineplot(data=ts_absorb_ratio, linewidth=3)
plt.title('Absorption Ratio via PCA')
#plt.savefig("Absorption_Ratio_SPX.png", dpi=900)

## Deriving a trading strategy from the absorption ratio according to the Kritzman paper
Having computed daily (this means the step size is 1) Absorption Ratio times series, we further follow M. Kritzman to make use of AR to define yet another measure: AR Delta. In particular:
$$ AR\delta = \frac{AR_{15d} - AR_{1y}}{ AR\sigma_{1y}}$$
We use  $AR\delta$ to build simple portfolio trading strategy

In [ ]:
# following Kritzman and computing AR_delta = (15d_AR -1yr_AR) / sigma_AR
ts_ar = ts_absorb_ratio
ar_mean_1yr = ts_ar.rolling(252).mean()
ar_mean_15d = ts_ar.rolling(15).mean()
ar_sd_1yr = ts_ar.rolling(252).std()
ar_delta = (ar_mean_15d - ar_mean_1yr) / ar_sd_1yr    # standardized shift in absorption ratio

df_plot = pd.DataFrame({'AR_delta': ar_delta.values, 'AR_1yr': ar_mean_1yr.values, 'AR_15d': ar_mean_15d.values}, 
                       index=ts_ar.index)
df_plot = df_plot.dropna()
if df_plot.shape[0] > 0:
    sns.lineplot(data=df_plot, linewidth=3)
    plt.title("Absorption Ratio Delta")

#### Part 3 (AR Delta Trading Strategy)

The AR Delta trading strategy forms a portfolio of EQ and FI, following these simple rules:

* __$ -1\sigma < AR < +1\sigma $__	 50 / 50 weights for EQ / FI
* __$ AR > +1\sigma $__	             0 / 100 weights for EQ / FI
* __$ AR < -1\sigma $__	             100 / 0 weights for EQ / FI

Here we compute AR Delta strategy weights using data from the same data set. As expected, the average number of trades per year is very low.

In [ ]:
def get_weight(ar_delta):
    '''
    Calculate EQ / FI portfolio weights based on Absorption Ratio delta
    Arguments:
    ar_delta -- Absorption Ratio delta
    
    Return: 
        wgts -- a vector of portfolio weights
    '''
    if ar_delta > 1: return [0.0,1.0]
    if ar_delta < -1: return [1.0, 0.0]
    return [0.5, 0.5]

In [ ]:
ar_delta_data = ar_delta[251:]

rebal_dates = np.zeros(len(ar_delta_data))
wgts = pd.DataFrame(data=np.zeros((len(ar_delta_data.index), 2)), index=ar_delta_data.index, columns=('EQ', 'FI'))

prtf_wgts = get_weight(ar_delta_data.values[0])
wgts.iloc[0, :] = prtf_wgts
for ix in range(1, len(ar_delta_data)):
    prtf_wgts = get_weight(ar_delta_data.values[ix])
    wgts.iloc[ix, :] = prtf_wgts
    if wgts.iloc[ix-1, :][0] != prtf_wgts[0]:
        rebal_dates[ix] = 1

ts_rebal_dates = pd.Series(rebal_dates, index=ar_delta_data.index)
ts_trades_per_year = ts_rebal_dates.groupby([ts_rebal_dates.index.year]).sum()
print('Average number of trades per year %.2f' % ts_trades_per_year.mean())
display(wgts.tail())

#### Calculate performance of backtested strategy


In [ ]:
def backtest_strategy(strat_wgts, asset_returns, periods_per_year = 252):
    '''
    Calculate portfolio returns and return portfolio strategy performance
    Arguments:
    
    strat_wgts -- pandas.DataFrame of weights of the assets
    asset_returns -- pandas.DataFrame of asset returns
    periods_per_year -- number of return observations per year
    
    Return: 
        (ann_ret, ann_vol, sharpe) -- a tuple of (annualized return, annualized volatility, sharpe ratio)
    '''
    
    together = pd.merge(strat_wgts, asset_returns, left_index=True, right_index=True)
    together["Returns"] = together["EQ"]*together["EQ_RETURN"]+together["FI"]*together["FI_RETURN"]
    annualized_return = np.prod(together.Returns.values + 1)**(periods_per_year/len(together.Returns))-1
    annualized_volatility = together.Returns.std() * np.sqrt(periods_per_year)
    return annualized_return, annualized_volatility, annualized_return/annualized_volatility

In [ ]:
ann_ret, ann_vol, sharpe = backtest_strategy(wgts, etf_r)
print('Absorption Ratio strategy:', ann_ret, ann_vol, sharpe)
eq_wgts = wgts.copy()
eq_wgts.iloc[:, ] = 0.5
ann_ret_eq_wgt, ann_vol_eq_wgt, sharpe_eq_wgt = backtest_strategy(eq_wgts, etf_r)
print('Equally weighted:', ann_ret_eq_wgt, ann_vol_eq_wgt, sharpe_eq_wgt)